## HW1 ASSIGNMENT 2

### OBJECTIVES

* Perform basic data engineering and visualization in Python using an external data set.
* Perform data engineering in Python using OpenAQ API.
* Perform basic data analysis in Python using OpenAQ weather data.
* Perform basic statistical significance tests of air quality data.
* Complete the online HW1 assessment.
* BONUS: Build and analyze a more complex OpenAQ data set. 

## Perfom basic data engineering and visualization in Python using an external set data.

In [1]:
# IMPORTING LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time
import json 
import os
import folium 

### § Task: Use Python to make HTTP/API calls to obtain and prepare data.

In [2]:
## Function to obtain the list of all US airports
def get_airport_data(url):
    # Read the data into the data frame
    airport_data = pd.read_csv(url)
    # Save data frame to a CSV file 
    airport_data.to_csv('all_us_airports.csv', index=False)

    print("Data saved successfully")

data_url = "https://ourairports.com/countries/US/airports.csv"
get_airport_data(data_url)

Data saved successfully


In [3]:
US_airport = pd.read_csv("all_us_airports.csv")
US_airport

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,...,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated
0,3632,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,125.0,NaN,United States,US,...,Los Angeles,1,KLAX,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,Tom Bradley,1335475,2023-12-21T12:31:02+00:00
1,3754,KORD,large_airport,Chicago O'Hare International Airport,41.978600,-87.904800,672.0,NaN,United States,US,...,Chicago,1,KORD,ORD,ORD,https://www.flychicago.com/ohare/home/pages/de...,https://en.wikipedia.org/wiki/O'Hare_Internati...,"CHI, Orchard Place",1503175,2018-09-16T02:35:35+00:00
2,3622,KJFK,large_airport,John F Kennedy International Airport,40.639447,-73.779317,13.0,NaN,United States,US,...,New York,1,KJFK,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild, IDL, ...",1052075,2022-10-18T18:49:55+00:00
3,3384,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,1026.0,NaN,United States,US,...,Atlanta,1,KATL,ATL,ATL,http://www.atlanta-airport.com/,https://en.wikipedia.org/wiki/Hartsfield–Jacks...,NaN,2002475,2018-09-19T14:50:01+00:00
4,3878,KSFO,large_airport,San Francisco International Airport,37.618999,-122.375000,13.0,NaN,United States,US,...,San Francisco,1,KSFO,SFO,SFO,http://www.flysfo.com/,https://en.wikipedia.org/wiki/San_Francisco_In...,"QSF, QBA",1112475,2008-06-13T14:30:04+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30913,353111,US-6578,closed,Kettenpom Airport,40.160970,-123.463850,3408.0,NaN,United States,US,...,Zenia,0,NaN,NaN,NaN,NaN,NaN,NaN,0,2022-02-28T15:05:57+00:00
30914,322813,US-0527,heliport,AdventHealth Zephyrhills Heliport,28.261934,-82.185663,160.0,NaN,United States,US,...,Zephyrhills,0,FD75,NaN,FD75,NaN,NaN,Florida Hospital Zephyrhills Heliport,0,2021-06-11T17:55:06+00:00
30915,324916,US-0652,heliport,Rotary Run Heliport,40.605805,-81.415002,1027.0,NaN,United States,US,...,Zoar,0,NaN,NaN,0OH2,NaN,NaN,NaN,0,2017-09-09T20:04:27+00:00
30916,327838,US-0874,heliport,Black Rock Heliport,35.084975,-108.788350,6454.0,NaN,United States,US,...,Zuni,0,NaN,NaN,NM1,NaN,NaN,NaN,0,2018-05-15T23:12:01+00:00


In [4]:
US_airport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30918 entries, 0 to 30917
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 30918 non-null  int64  
 1   ident              30918 non-null  object 
 2   type               30918 non-null  object 
 3   name               30918 non-null  object 
 4   latitude_deg       30918 non-null  float64
 5   longitude_deg      30918 non-null  float64
 6   elevation_ft       28914 non-null  float64
 7   continent          110 non-null    object 
 8   country_name       30918 non-null  object 
 9   iso_country        30918 non-null  object 
 10  region_name        30918 non-null  object 
 11  iso_region         30918 non-null  object 
 12  local_region       30918 non-null  object 
 13  municipality       30860 non-null  object 
 14  scheduled_service  30918 non-null  int64  
 15  gps_code           20304 non-null  object 
 16  iata_code          197

In [5]:
# Checking for null values 
US_airport.isnull().sum()

id                       0
ident                    0
type                     0
name                     0
latitude_deg             0
longitude_deg            0
elevation_ft          2004
continent            30808
country_name             0
iso_country              0
region_name              0
iso_region               0
local_region             0
municipality            58
scheduled_service        0
gps_code             10614
iata_code            28947
local_code            9689
home_link            29759
wikipedia_link       28329
keywords             24774
score                    0
last_updated             0
dtype: int64

In [6]:
# We can see that the continent has more NAN values so we are going to remove the column 
US_airport = US_airport.drop('continent', axis = 1)

In [7]:
# We are also removing the non-essential column from the data frame.
# Like id, iso_country, iso_region, local_region, country_name, last_updated, home link, wikipedia link
US_airport = US_airport.drop(['id', 'country_name','iso_country', 'iso_region', 'local_region', 'home_link', 'wikipedia_link'], axis = 1)

In [8]:
# Removing NAN value from dataset 
US_airport = US_airport.dropna(subset = ['iata_code'])

In [9]:
US_airport.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1971 entries, 0 to 30906
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ident              1971 non-null   object 
 1   type               1971 non-null   object 
 2   name               1971 non-null   object 
 3   latitude_deg       1971 non-null   float64
 4   longitude_deg      1971 non-null   float64
 5   elevation_ft       1941 non-null   float64
 6   region_name        1971 non-null   object 
 7   municipality       1971 non-null   object 
 8   scheduled_service  1971 non-null   int64  
 9   gps_code           1896 non-null   object 
 10  iata_code          1971 non-null   object 
 11  local_code         1957 non-null   object 
 12  keywords           483 non-null    object 
 13  score              1971 non-null   int64  
 14  last_updated       1971 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 246.4+ KB


### § Task: Filter data to a subset for further use.

In [10]:
def filter_large_airports():
    # Load the data from the CSV file
    filtered_data = pd.read_csv('all_us_airports.csv')

    # Filter the data to include only large airports
    filtered_data = filtered_data[filtered_data['type'] == 'large_airport']

    # Select only the relevant columns
    filtered_data = filtered_data[['name', 'latitude_deg', 'longitude_deg', 'iata_code']]

    # Save the filtered data to a new CSV file
    filtered_data.to_csv('large_us_airports.csv', index=False)

    print("Data filtered and saved successfully!")

# Call the function
filter_large_airports()

Data filtered and saved successfully!


In [11]:
large_airport_data = pd.read_csv("large_us_airports.csv")
large_airport_data

,name,latitude_deg,longitude_deg,iata_code
0,Los Angeles International Airport,33.942501,-118.407997,LAX
1,Chicago O'Hare International Airport,41.978600,-87.904800,ORD
2,John F Kennedy International Airport,40.639447,-73.779317,JFK
3,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,ATL
4,San Francisco International Airport,37.618999,-122.375000,SFO
...,...,...,...,...
62,Portland International Jetport,43.646198,-70.309303,PWM
63,Orlando Sanford International Airport,28.777599,-81.237503,SFB
64,Fairbanks International Airport,64.815102,-147.856003,FAI
65,Camarillo International Airport,34.213699,-119.094002,NaN


In [12]:
large_airport_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           67 non-null     object 
 1   latitude_deg   67 non-null     float64
 2   longitude_deg  67 non-null     float64
 3   iata_code      66 non-null     object 
dtypes: float64(2), object(2)
memory usage: 2.2+ KB


In [13]:
# One of the columns has one NaN value which we will ignore by removing it 
rows_with_nan = large_airport_data[large_airport_data.isna().any(axis = 1)]
print(rows_with_nan)

                               name  latitude_deg  longitude_deg iata_code
65  Camarillo International Airport     34.213699    -119.094002       NaN


In [14]:
# Removing the row with nan value 
large_airport_data = large_airport_data.drop(rows_with_nan.index)
large_airport_data

,name,latitude_deg,longitude_deg,iata_code
0,Los Angeles International Airport,33.942501,-118.407997,LAX
1,Chicago O'Hare International Airport,41.978600,-87.904800,ORD
2,John F Kennedy International Airport,40.639447,-73.779317,JFK
3,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,ATL
4,San Francisco International Airport,37.618999,-122.375000,SFO
...,...,...,...,...
61,Tulsa International Airport,36.198399,-95.888100,TUL
62,Portland International Jetport,43.646198,-70.309303,PWM
63,Orlando Sanford International Airport,28.777599,-81.237503,SFB
64,Fairbanks International Airport,64.815102,-147.856003,FAI


### § Task: Plot the data using Folium

In [15]:
# Installing Folium 
!pip install folium

In [16]:
# Plotting the map 
latitude_longitude = [large_airport_data['latitude_deg'].mean(), large_airport_data['longitude_deg'].mean()]
map = folium.Map(
    location = latitude_longitude,
    zoom_control = 4,
)
# Iterating through the dataset 
for index, row in large_airport_data.iterrows():
    angle = 315
    popup_content = f"{row['name']} ({row['iata_code']})"
    icon_params = {"prefix": "fa", "icon": "plane"}
    icon = folium.Icon(angle = angle, **icon_params)
    folium.Marker(location = [row['latitude_deg'], row['longitude_deg']],
                  popup = popup_content,
                  icon = icon).add_to(map)

map.save('large_airport_map.html')
map

## Perform data engineering in Python using OpenAQ API.

### § Task: Obtain a dataset from OpenAQ

In [17]:
# Define the base URL for the OpenAQ API
base_url = "https://api.openaq.org/v2/measurements"

# Define parameters for the API request
params = {
    "date_from": "2023-06-06T00:00:00Z",
    "date_to": "2023-06-06T23:59:59Z",
    "coordinates": "42.33143000,-83.04575000",
    "radius": 7500,
    "parameter_id": 2,
    "limit": 10000
}

# Define headers for the API request
headers = {"accept": "application/json"}

# Send a GET request to the OpenAQ API
response = requests.get(base_url, params = params, headers = headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # Save the data to a JSON file
    with open('openaq_data.json', 'w') as f:
        json.dump(data, f)
else:
    print(f"Request failed with status code {response.status_code}")

# Pause for 1-2 seconds as a courtesy to the OpenAQ API
time.sleep(2)

In [18]:
with open('openaq_data.json', 'r') as json_file:
    # Load the JSON data from the file
    data = json.load(json_file)
    
original_openaq_data = pd.DataFrame(data['results'])
# Print the DataFrame
original_openaq_data

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,1385,Windsor Downtown,pm25,125.0,"{'utc': '2023-06-06T23:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 42.3144, 'longitude': -83.0433}",US,None,False,None,Governmental Organization,reference grade
1,1385,Windsor Downtown,pm25,21.0,"{'utc': '2023-06-06T22:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 42.3144, 'longitude': -83.0433}",US,None,False,None,Governmental Organization,reference grade
2,1385,Windsor Downtown,pm25,14.0,"{'utc': '2023-06-06T21:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 42.3144, 'longitude': -83.0433}",US,None,False,None,Governmental Organization,reference grade
3,1385,Windsor Downtown,pm25,18.0,"{'utc': '2023-06-06T20:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 42.3144, 'longitude': -83.0433}",US,None,False,None,Governmental Organization,reference grade
4,1385,Windsor Downtown,pm25,21.0,"{'utc': '2023-06-06T19:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 42.3144, 'longitude': -83.0433}",US,None,False,None,Governmental Organization,reference grade
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6279,7002,MILITARY PARK,pm25,33.7,"{'utc': '2023-06-06T05:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 42.312078, 'longitude': -83.103469}",US,None,False,None,Governmental Organization,reference grade
6280,7002,MILITARY PARK,pm25,31.7,"{'utc': '2023-06-06T04:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 42.312078, 'longitude': -83.103469}",US,None,False,None,Governmental Organization,reference grade
6281,7002,MILITARY PARK,pm25,29.9,"{'utc': '2023-06-06T03:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 42.312078, 'longitude': -83.103469}",US,None,False,None,Governmental Organization,reference grade
6282,7002,MILITARY PARK,pm25,28.8,"{'utc': '2023-06-06T02:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 42.312078, 'longitude': -83.103469}",US,None,False,None,Governmental Organization,reference grade


### § Task: Transform, filter and store the OpenAQ data as CSV

#### Transform

In [19]:
# Function to convert the coordinates field to two fields: sensor_lat and sensor_lon
def transform_openaq_data(data):
    openaq_data = pd.DataFrame(data['results'])
    openaq_data['sensor_lat'] = openaq_data['coordinates'].apply(lambda x: x['latitude'])
    openaq_data['sensor_lon'] = openaq_data['coordinates'].apply(lambda x: x['longitude'])
    openaq_data['local_time'] = pd.to_datetime(openaq_data['date'].apply(lambda x: x['local']))
    
    # Drop the original 'coordinate' column
    openaq_data.drop(columns=['date'], inplace = True)
    return openaq_data

transformed_data = transform_openaq_data(data)
transformed_data

,locationId,location,parameter,value,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType,sensor_lat,sensor_lon,local_time
0,1385,Windsor Downtown,pm25,125.0,µg/m³,"{'latitude': 42.3144, 'longitude': -83.0433}",US,None,False,None,Governmental Organization,reference grade,42.314400,-83.043300,2023-06-06 19:00:00-04:00
1,1385,Windsor Downtown,pm25,21.0,µg/m³,"{'latitude': 42.3144, 'longitude': -83.0433}",US,None,False,None,Governmental Organization,reference grade,42.314400,-83.043300,2023-06-06 18:00:00-04:00
2,1385,Windsor Downtown,pm25,14.0,µg/m³,"{'latitude': 42.3144, 'longitude': -83.0433}",US,None,False,None,Governmental Organization,reference grade,42.314400,-83.043300,2023-06-06 17:00:00-04:00
3,1385,Windsor Downtown,pm25,18.0,µg/m³,"{'latitude': 42.3144, 'longitude': -83.0433}",US,None,False,None,Governmental Organization,reference grade,42.314400,-83.043300,2023-06-06 16:00:00-04:00
4,1385,Windsor Downtown,pm25,21.0,µg/m³,"{'latitude': 42.3144, 'longitude': -83.0433}",US,None,False,None,Governmental Organization,reference grade,42.314400,-83.043300,2023-06-06 15:00:00-04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6279,7002,MILITARY PARK,pm25,33.7,µg/m³,"{'latitude': 42.312078, 'longitude': -83.103469}",US,None,False,None,Governmental Organization,reference grade,42.312078,-83.103469,2023-06-06 01:00:00-04:00
6280,7002,MILITARY PARK,pm25,31.7,µg/m³,"{'latitude': 42.312078, 'longitude': -83.103469}",US,None,False,None,Governmental Organization,reference grade,42.312078,-83.103469,2023-06-06 00:00:00-04:00
6281,7002,MILITARY PARK,pm25,29.9,µg/m³,"{'latitude': 42.312078, 'longitude': -83.103469}",US,None,False,None,Governmental Organization,reference grade,42.312078,-83.103469,2023-06-05 23:00:00-04:00
6282,7002,MILITARY PARK,pm25,28.8,µg/m³,"{'latitude': 42.312078, 'longitude': -83.103469}",US,None,False,None,Governmental Organization,reference grade,42.312078,-83.103469,2023-06-05 22:00:00-04:00


#### Filter

In [20]:
def filter_openaq_data(transformed_data):
    filtered_data = transformed_data[['locationId', 'location', 'entity', 'parameter', 'value', 'local_time', 'sensor_lat', 'sensor_lon']]
    
    # Filter the data to include only Governmental Organization and Community Organization
    filtered_data = filtered_data[filtered_data['entity'].isin(['Governmental Organization', 'Community Organization'])]
    return filtered_data

filtered_data = filter_openaq_data(transformed_data)
filtered_data

,locationId,location,entity,parameter,value,local_time,sensor_lat,sensor_lon
0,1385,Windsor Downtown,Governmental Organization,pm25,125.0,2023-06-06 19:00:00-04:00,42.314400,-83.043300
1,1385,Windsor Downtown,Governmental Organization,pm25,21.0,2023-06-06 18:00:00-04:00,42.314400,-83.043300
2,1385,Windsor Downtown,Governmental Organization,pm25,14.0,2023-06-06 17:00:00-04:00,42.314400,-83.043300
3,1385,Windsor Downtown,Governmental Organization,pm25,18.0,2023-06-06 16:00:00-04:00,42.314400,-83.043300
4,1385,Windsor Downtown,Governmental Organization,pm25,21.0,2023-06-06 15:00:00-04:00,42.314400,-83.043300
...,...,...,...,...,...,...,...,...
6279,7002,MILITARY PARK,Governmental Organization,pm25,33.7,2023-06-06 01:00:00-04:00,42.312078,-83.103469
6280,7002,MILITARY PARK,Governmental Organization,pm25,31.7,2023-06-06 00:00:00-04:00,42.312078,-83.103469
6281,7002,MILITARY PARK,Governmental Organization,pm25,29.9,2023-06-05 23:00:00-04:00,42.312078,-83.103469
6282,7002,MILITARY PARK,Governmental Organization,pm25,28.8,2023-06-05 22:00:00-04:00,42.312078,-83.103469


#### Store

In [21]:
# Save the filtered data to a new CSV file
filtered_data.to_csv('20230606_detroit_downtown_7_5km_aq.csv', index=False)
print("Data filtered and saved successfully!")

Data filtered and saved successfully!


In [22]:
# Load the data 
openaq_data = pd.read_csv('20230606_detroit_downtown_7_5km_aq.csv')
openaq_data

,locationId,location,entity,parameter,value,local_time,sensor_lat,sensor_lon
0,1385,Windsor Downtown,Governmental Organization,pm25,125.0,2023-06-06 19:00:00-04:00,42.314400,-83.043300
1,1385,Windsor Downtown,Governmental Organization,pm25,21.0,2023-06-06 18:00:00-04:00,42.314400,-83.043300
2,1385,Windsor Downtown,Governmental Organization,pm25,14.0,2023-06-06 17:00:00-04:00,42.314400,-83.043300
3,1385,Windsor Downtown,Governmental Organization,pm25,18.0,2023-06-06 16:00:00-04:00,42.314400,-83.043300
4,1385,Windsor Downtown,Governmental Organization,pm25,21.0,2023-06-06 15:00:00-04:00,42.314400,-83.043300
...,...,...,...,...,...,...,...,...
6279,7002,MILITARY PARK,Governmental Organization,pm25,33.7,2023-06-06 01:00:00-04:00,42.312078,-83.103469
6280,7002,MILITARY PARK,Governmental Organization,pm25,31.7,2023-06-06 00:00:00-04:00,42.312078,-83.103469
6281,7002,MILITARY PARK,Governmental Organization,pm25,29.9,2023-06-05 23:00:00-04:00,42.312078,-83.103469
6282,7002,MILITARY PARK,Governmental Organization,pm25,28.8,2023-06-05 22:00:00-04:00,42.312078,-83.103469


### Perfom basic data analysis in Python using OpenAQ weather data.

In [23]:
openaq_data.isnull().sum()

locationId    0
location      0
entity        0
parameter     0
value         0
local_time    0
sensor_lat    0
sensor_lon    0
dtype: int64

#### § Task: Load your Detroit data and answer the following questions

In [24]:
# 1. What is the mean and median PM2.5 reading over all sensors?
PM25_Mean = openaq_data['value'].mean()
PM25_Median = openaq_data['value'].median()
print("The mean PM2.5 reading is: ",PM25_Mean)
print("The median PM2.5 reading is: ",PM25_Median)

# 2. What is the standard deviation?
Standard_deviation = openaq_data['value'].std()
print("The standard deviation reading is: ",Standard_deviation)

# 3. Which location_id recorded the highest PM2.5? What was the reading?
highest_PM25_location_id = openaq_data.nlargest(1, 'value')['locationId'].iloc[0]
highest_PM25 = openaq_data.nlargest(1, 'value')['value'].iloc[0]
print("Location ID with the highest PM2.5 reading:", highest_PM25_location_id)
print("Highest PM2.5 reading:", highest_PM25)

# 4. What is the ratio of Community Organization to Governmental Organization entity type?
ratio = openaq_data['entity'].value_counts(normalize = True)['Community Organization'] / openaq_data['entity'].value_counts(normalize = True)['Governmental Organization']
print("The ratio of Community Organization to Governmental Organization entity type", ratio)

# 5. How many unique sensor stations are in the data (use sensorId)?
unique_sensor_station = openaq_data['locationId'].nunique()
print("The number of unique sensor sttaions are: ", unique_sensor_station)

# 6. What is the station density per km?

# 7. What is the daily mean, median, min, max, 75%, and standard deviation separately for all Community Organization and Government Organization sensors in the data (that is group each separately and report the statistics being asked)?

# 8. What is your opinion of the differences in the statistics? Comment specifically about the mean and 75%.

The mean PM2.5 reading is:  66.0975970719287
The median PM2.5 reading is:  37.05
The standard deviation reading is:  216.46080063153576
Location ID with the highest PM2.5 reading: 65949
Highest PM2.5 reading: 1759.5
The ratio of Community Organization to Governmental Organization entity type 67.30434782608695
The number of unique sensor sttaions are:  15
